# 03. Pandas 집계 분석 - 실습 문제

## 실습 안내
- 총 10개 문제
- 설비별, 제품별, 시간대별 집계 분석
- groupby, pivot_table, crosstab 활용
- 실제 제조 현장의 분석 시나리오

## 데이터 로드 및 전처리

In [91]:
import pandas as pd
import numpy as np

# 데이터 불러오기
production_df = pd.read_csv('../data/05_production.csv', encoding='utf-8-sig')
quality_df = pd.read_csv('../data/07_quality_inspection.csv', encoding='utf-8-sig', na_values=['\\N'])
equipment_df = pd.read_csv('../data/01_equipment.csv', encoding='utf-8-sig')
operation_df = pd.read_csv('../data/06_equipment_operation.csv', encoding='utf-8-sig')

# 기본 전처리
production_df['production_date'] = pd.to_datetime(production_df['production_date'])
production_df['defect_rate'] = (production_df['defect_quantity'] / production_df['actual_quantity'] * 100).round(2)
quality_df['inspection_time'] = pd.to_datetime(quality_df['inspection_time'])

print("데이터 로드 완료!")
print(f"생산: {len(production_df):,}건")
print(f"품질: {len(quality_df):,}건")
print(f"설비: {len(equipment_df):,}건")
print(f"설비운영: {len(operation_df):,}건")

데이터 로드 완료!
생산: 1,872건
품질: 37,417건
설비: 5건
설비운영: 3,304건


---
## 문제 1: 설비별 생산 통계

**시나리오**: 각 설비의 생산 성과를 종합적으로 분석하세요.

**요구사항**:
1. 설비별로 다음 지표 계산:
   - 생산 건수 (count)
   - 총 생산량 (actual_quantity의 sum)
   - 평균 생산량 (actual_quantity의 mean)
   - 총 불량수 (defect_quantity의 sum)
   - 평균 불량률 (defect_rate의 mean)
2. 총 생산량 기준 내림차순 정렬
3. 소수점 2자리로 반올림



In [ ]:
# 1. 설비별로 다음 지표 계산:
#    - 생산 건수 (count)
#    - 총 생산량 (actual_quantity의 sum)
#    - 평균 생산량 (actual_quantity의 mean)
#    - 총 불량수 (defect_quantity의 sum)
#    - 평균 불량률 (defect_rate의 mean)
production_agg_eq = production_df.groupby('equipment_id').agg(생산_건수=('production_id', 'count'),
                                                              총_생산량=('actual_quantity', 'sum'),
                                                              평균_생산량=('actual_quantity', 'mean'),
                                                              총_불량수=('defect_quantity', 'sum'),
                                                              평균_불량률=('defect_rate', 'mean'))
production_agg_eq

,생산_건수,총_생산량,평균_생산량,총_불량수,평균_불량률
equipment_id,,,,,
ASM-001,234,22485,96.089744,2726,12.116453
INJ-001,262,28163,107.492366,3017,10.759046
INJ-002,430,51958,120.832558,4504,8.701302
PRESS-001,468,52069,111.258547,5123,9.910855
PRESS-002,478,51929,108.638075,5506,10.675565


In [93]:
# 2. 총 생산량 기준 내림차순 정렬
production_agg_eq.sort_values('총_생산량', ascending=False, inplace=True)
production_agg_eq

,생산_건수,총_생산량,평균_생산량,총_불량수,평균_불량률
equipment_id,,,,,
PRESS-001,468,52069,111.258547,5123,9.910855
INJ-002,430,51958,120.832558,4504,8.701302
PRESS-002,478,51929,108.638075,5506,10.675565
INJ-001,262,28163,107.492366,3017,10.759046
ASM-001,234,22485,96.089744,2726,12.116453


In [94]:
# 3. 소수점 2자리로 반올림
production_agg_eq = production_agg_eq.round({'평균_생산량':2, '평균_불량률':2})
production_agg_eq

,생산_건수,총_생산량,평균_생산량,총_불량수,평균_불량률
equipment_id,,,,,
PRESS-001,468,52069,111.26,5123,9.91
INJ-002,430,51958,120.83,4504,8.70
PRESS-002,478,51929,108.64,5506,10.68
INJ-001,262,28163,107.49,3017,10.76
ASM-001,234,22485,96.09,2726,12.12


---
## 문제 2: 제품별 품질 분석

**시나리오**: 제품별 품질 수준을 파악하여 문제가 있는 제품을 찾으세요.

**요구사항**:
1. 제품별로 다음 지표 계산:
   - 생산 건수
   - 총 생산량
   - 평균 불량률
   - 최대 불량률
   - 최소 불량률
2. 평균 불량률이 높은 순서로 정렬



In [95]:
# 1. 제품별로 다음 지표 계산:
#    - 생산 건수
#    - 총 생산량
#    - 평균 불량률
#    - 최대 불량률
#    - 최소 불량률
production_agg_pd = production_df.groupby('product_code').agg(생산_건수=('production_id', 'count'),
                                                              총_생산량=('actual_quantity', 'sum'),
                                                              평균_불량률=('defect_rate', 'mean'),
                                                              최대_불량률=('defect_rate', 'max'),
                                                              최소_불량률=('defect_rate', 'min'),).round({'평균_불량률':2})
production_agg_pd

,생산_건수,총_생산량,평균_불량률,최대_불량률,최소_불량률
product_code,,,,,
BUMPER-A,648,71135,10.38,19.48,2.0
DASH-C,583,64741,10.12,19.35,2.0
DOOR-B,641,70728,10.16,20.00,2.0


In [96]:
# 2. 평균 불량률이 높은 순서로 정렬
production_agg_pd = production_agg_pd.sort_values('평균_불량률', ascending=False)
production_agg_pd

,생산_건수,총_생산량,평균_불량률,최대_불량률,최소_불량률
product_code,,,,,
BUMPER-A,648,71135,10.38,19.48,2.0
DOOR-B,641,70728,10.16,20.00,2.0
DASH-C,583,64741,10.12,19.35,2.0


---
## 문제 3: 교대조별 생산 효율 비교

**시나리오**: 주간조와 야간조의 생산 효율을 비교 분석하세요.

**요구사항**:
1. 교대조(shift)별로 다음 지표 계산:
   - 생산 건수
   - 평균 생산량
   - 평균 불량률
   - 평균 사이클 타임
   - 목표 달성률 평균 (actual_quantity / target_quantity * 100)
2. 결과 해석: 어느 교대조가 더 효율적인가?



In [98]:
# 1. 교대조(shift)별로 다음 지표 계산:
#    - 생산 건수
#    - 평균 생산량
#    - 평균 불량률
#    - 평균 사이클 타임
#    - 목표 달성률 평균 (actual_quantity / target_quantity * 100)
production_df['목표_달성률'] = (production_df['actual_quantity']/production_df['target_quantity']*100).round(2)
production_agg_shift = production_df.groupby('shift').agg(생산_건수=('production_id', 'count'),
                                                          평균_생산량=('actual_quantity', 'mean'),
                                                          평균_불량률=('defect_rate', 'mean'),
                                                          평균_사이클_타임=('cycle_time', 'mean'),
                                                          목표_달성률_평균=('목표_달성률', 'mean')).round({'평균_생산량':2,
                                                                                                       '평균_불량률':2,
                                                                                                       '평균_사이클_타임':2,
                                                                                                       '목표_달성률_평균':2})
production_agg_shift

,생산_건수,평균_생산량,평균_불량률,평균_사이클_타임,목표_달성률_평균
shift,,,,,
DAY,936,109.71,8.82,76.64,94.82
NIGHT,936,111.02,11.63,76.64,96.64


2. 결과 해석: 어느 교대조가 더 효율적인가?

- 야간조의 평균 생산량과 목표 달성률 평균이 높은 것을 통해 생산 효율이 좋은 것을 확인할 수 있다. 다만, 평균 불량률이 비교적 높다.

---
## 문제 4: 설비 + 제품 복합 분석

**시나리오**: 각 설비가 어떤 제품을 얼마나 생산하는지 분석하세요.

**요구사항**:
1. 설비 + 제품별로 다음 집계:
   - 생산 건수
   - 총 생산량
   - 평균 불량률
2. 멀티인덱스 결과에서 특정 설비(예: INJ-001)의 제품별 데이터만 추출



In [99]:
# 1. 설비 + 제품별로 다음 집계:
#    - 생산 건수
#    - 총 생산량
#    - 평균 불량률

production_agg_eqpd = production_df.groupby(['equipment_id', 'product_code']).agg(생산_건수=('production_id', 'count'),
                                                                                  총_생산량=('actual_quantity', 'sum'),
                                                                                  평균_불량률=('defect_rate', 'mean')).round({'평균_불량률':2})
production_agg_eqpd

생산_건수  총_생산량  평균_불량률
equipment_id product_code                      
ASM-001      BUMPER-A         74   6804   11.10
             DASH-C           70   6779   12.51
             DOOR-B           90   8902   12.65
INJ-001      BUMPER-A        104  10858   11.51
             DASH-C           85   9536   10.19
             DOOR-B           73   7769   10.35
INJ-002      BUMPER-A        147  18053    9.53
             DASH-C          122  14766    8.11
             DOOR-B          161  19139    8.39
PRESS-001    BUMPER-A        171  18757    9.52
             DASH-C          139  15670   10.21
             DOOR-B          158  17642   10.07
PRESS-002    BUMPER-A        152  16663   11.02
             DASH-C          167  17990   10.48
             DOOR-B          159  17276   10.55

In [100]:
# 2. 멀티인덱스 결과에서 특정 설비(예: INJ-001)의 제품별 데이터만 추출
production_agg_eqpd.loc['ASM-001']

,생산_건수,총_생산량,평균_불량률
product_code,,,
BUMPER-A,74,6804,11.10
DASH-C,70,6779,12.51
DOOR-B,90,8902,12.65


---
## 문제 5: 피벗 테이블 - 설비 x 제품 생산량 매트릭스

**시나리오**: 설비와 제품의 조합별 생산량을 한눈에 보는 표를 만드세요.

**요구사항**:
1. 피벗 테이블 생성:
   - 행: equipment_id
   - 열: product_code
   - 값: actual_quantity의 합계
   - 결측치는 0으로 채우기
2. 행/열 총계 추가 (margins=True)



In [101]:
# 1. 피벗 테이블 생성:
#    - 행: equipment_id
#    - 열: product_code
#    - 값: actual_quantity의 합계
#    - 결측치는 0으로 채우기
pd.pivot_table(production_df, index='equipment_id', columns='product_code', values='actual_quantity', aggfunc='sum', fill_value=0)

product_code,BUMPER-A,DASH-C,DOOR-B
equipment_id,,,
ASM-001,6804,6779,8902
INJ-001,10858,9536,7769
INJ-002,18053,14766,19139
PRESS-001,18757,15670,17642
PRESS-002,16663,17990,17276


In [102]:
# 2. 행/열 총계 추가 (margins=True)
pd.pivot_table(production_df, index='equipment_id', columns='product_code', values='actual_quantity',
               aggfunc='sum', fill_value=0, margins=True)

product_code,BUMPER-A,DASH-C,DOOR-B,All
equipment_id,,,,
ASM-001,6804,6779,8902,22485
INJ-001,10858,9536,7769,28163
INJ-002,18053,14766,19139,51958
PRESS-001,18757,15670,17642,52069
PRESS-002,16663,17990,17276,51929
All,71135,64741,70728,206604


---
## 문제 6: 피벗 테이블 - 설비 x 교대조 평균 불량률

**시나리오**: 설비별로 교대조에 따라 불량률이 어떻게 다른지 파악하세요.

**요구사항**:
1. 피벗 테이블 생성:
   - 행: equipment_id
   - 열: shift
   - 값: defect_rate의 평균
2. 소수점 2자리 반올림
3. 주간과 야간의 불량률 차이가 큰 설비 찾기



In [103]:
# 1. 피벗 테이블 생성:
#    - 행: equipment_id
#    - 열: shift
#    - 값: defect_rate의 평균

eq_shift_pivot = pd.pivot_table(production_df, index='equipment_id', columns='shift', values='defect_rate', aggfunc='mean')
eq_shift_pivot

shift,DAY,NIGHT
equipment_id,,
ASM-001,10.671282,13.561624
INJ-001,9.356031,12.162061
INJ-002,7.202512,10.200093
PRESS-001,8.529872,11.291838
PRESS-002,9.342469,12.008661


In [104]:
# 2. 소수점 2자리 반올림
eq_shift_pivot = eq_shift_pivot.round(2)
eq_shift_pivot

shift,DAY,NIGHT
equipment_id,,
ASM-001,10.67,13.56
INJ-001,9.36,12.16
INJ-002,7.20,10.20
PRESS-001,8.53,11.29
PRESS-002,9.34,12.01


In [105]:
# 3. 주간과 야간의 불량률 차이가 큰 설비 찾기
(eq_shift_pivot['DAY'] - eq_shift_pivot['NIGHT']).round(2).abs().sort_values(ascending=False)

equipment_id
INJ-002      3.00
ASM-001      2.89
INJ-001      2.80
PRESS-001    2.76
PRESS-002    2.67
dtype: float64

---
## 문제 7: 불량 유형 분석 (crosstab)

**시나리오**: 제품별로 어떤 불량 유형이 많이 발생하는지 분석하세요.

**요구사항**:
1. 품질검사 데이터에서 불량(result='FAIL')만 필터링
2. 제품(product_code) x 불량코드(defect_code) 교차표 생성
3. 총계 포함
4. 각 제품의 주요 불량 유형 파악

**힌트**: 필터링 후 `pd.crosstab()`, margins=True

In [106]:
# 1. 품질검사 데이터에서 불량(result='FAIL')만 필터링
quality_fail = quality_df[quality_df['result'] == 'FAIL']
quality_fail

,inspection_id,production_id,equipment_id,product_code,inspection_time,inspection_type,result,defect_code,measurement_value,measurement_unit,inspector_id,lot_no,sample_size,notes,created_at
7,8,1,INJ-001,BUMPER-A,2024-01-01 08:26:47,FINAL,FAIL,D002,305.2058,mm,OP007,LOT2024010100101,1,NaN,2026-01-30 01:24:59
8,9,1,INJ-001,BUMPER-A,2024-01-01 09:30:37,FINAL,FAIL,D002,314.6991,mm,OP008,LOT2024010100101,1,NaN,2026-01-30 01:24:59
9,10,1,INJ-001,BUMPER-A,2024-01-01 08:44:05,FINAL,FAIL,D003,293.3244,mm,OP007,LOT2024010100101,1,NaN,2026-01-30 01:24:59
10,11,1,INJ-001,BUMPER-A,2024-01-01 08:35:47,FINAL,FAIL,D006,287.1234,mm,OP007,LOT2024010100101,1,NaN,2026-01-30 01:24:59
18,19,2,INJ-001,BUMPER-A,2024-01-01 21:53:40,FINAL,FAIL,D001,291.1185,mm,OP008,LOT2024010100110,1,NaN,2026-01-30 01:24:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37412,37413,1872,ASM-001,DOOR-B,2024-03-31 21:13:26,FINAL,FAIL,D002,608.2289,mm,OP008,LOT2024033100110,1,NaN,2026-01-30 01:25:02
37413,37414,1872,ASM-001,DOOR-B,2024-03-31 20:50:20,FINAL,FAIL,D001,609.7039,mm,OP007,LOT2024033100110,1,NaN,2026-01-30 01:25:02
37414,37415,1872,ASM-001,DOOR-B,2024-03-31 20:23:22,FINAL,FAIL,D010,590.4729,mm,OP008,LOT2024033100110,1,NaN,2026-01-30 01:25:02
37415,37416,1872,ASM-001,DOOR-B,2024-03-31 20:44:08,FINAL,FAIL,D001,609.0846,mm,OP008,LOT2024033100110,1,NaN,2026-01-30 01:25:02


In [107]:
# 2. 제품(product_code) x 불량코드(defect_code) 교차표 생성
pd.crosstab(index=quality_fail['product_code'], columns=quality_fail['defect_code'])

defect_code,D001,D002,D003,D004,D005,D006,D007,D008,D009,D010
product_code,,,,,,,,,,
BUMPER-A,1061,1458,720,385,1063,763,390,379,732,342
DASH-C,656,963,984,339,1325,948,334,182,460,304
DOOR-B,2829,1034,748,340,734,354,205,129,358,357


In [108]:
# 3. 총계 포함
pd.crosstab(index=quality_fail['product_code'], columns=quality_fail['defect_code'], margins=True)

defect_code,D001,D002,D003,D004,D005,D006,D007,D008,D009,D010,All
product_code,,,,,,,,,,,
BUMPER-A,1061,1458,720,385,1063,763,390,379,732,342,7293
DASH-C,656,963,984,339,1325,948,334,182,460,304,6495
DOOR-B,2829,1034,748,340,734,354,205,129,358,357,7088
All,4546,3455,2452,1064,3122,2065,929,690,1550,1003,20876


In [109]:
# 4. 각 제품의 주요 불량 유형 파악
pd.crosstab(index=quality_fail['product_code'], columns=quality_fail['defect_code']).idxmax(axis=1)

product_code
BUMPER-A    D002
DASH-C      D005
DOOR-B      D001
dtype: object

---
## 문제 8: 월별 생산 추이 분석

**시나리오**: 월별 생산량과 품질 추이를 분석하여 트렌드를 파악하세요.

**요구사항**:
1. production_date에서 년-월 추출 (dt.to_period('M'))
2. 월별로 다음 집계:
   - 생산 건수
   - 총 생산량
   - 평균 생산량
   - 평균 불량률
3. 시간 순서로 정렬
4. 처음 12개월 데이터 출력



In [110]:
# 1. production_date에서 년-월 추출 (dt.to_period('M'))
production_df['year-month'] = production_df['production_date'].dt.to_period('M')
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at,defect_rate,목표_달성률,year-month
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,4.94,83.51,2024-01
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,7.69,93.98,2024-01
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.22,90.60,2024-01
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.17,92.00,2024-01
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,5.43,104.88,2024-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,77.63,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.36,96.00,2024-03
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,76.15,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,16.15,95.59,2024-03
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,69.95,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.50,95.24,2024-03
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,90.10,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,16.53,84.62,2024-03


In [115]:
# 2. 월별로 다음 집계:
#    - 생산 건수
#    - 총 생산량
#    - 평균 생산량
#    - 평균 불량률
yearmonth_agg = production_df.groupby('year-month').agg(생산_건수=('production_id', 'count'),
                                                        총_생산량=('actual_quantity', 'sum'),
                                                        평균_생산량=('actual_quantity', 'mean')).round({'평균_생산량':2})
yearmonth_agg

,생산_건수,총_생산량,평균_생산량
year-month,,,
2024-01,626,69849,111.58
2024-02,602,66390,110.28
2024-03,644,70365,109.26


In [116]:
# 3. 시간 순서로 정렬
yearmonth_agg.sort_index()

,생산_건수,총_생산량,평균_생산량
year-month,,,
2024-01,626,69849,111.58
2024-02,602,66390,110.28
2024-03,644,70365,109.26


In [ ]:
# 4. 처음 12개월 데이터 출력
production_df[(production_df['production_date'] - production_df.loc[0, 'production_date']).dt.days <= 365]

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at,defect_rate,목표_달성률,year-month
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,4.94,83.51,2024-01
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,7.69,93.98,2024-01
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.22,90.60,2024-01
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.17,92.00,2024-01
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,5.43,104.88,2024-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,77.63,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.36,96.00,2024-03
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,76.15,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,16.15,95.59,2024-03
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,69.95,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.50,95.24,2024-03
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,90.10,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,16.53,84.62,2024-03


---
## 문제 9: Transform을 이용한 설비별 표준화

**시나리오**: 각 설비의 생산량을 설비별 평균과 비교하여 성과를 평가하세요.

**요구사항**:
1. 설비별 평균 생산량을 계산하여 새 컬럼으로 추가 (transform)
2. 설비별 표준편차를 계산하여 새 컬럼으로 추가
3. 각 생산 건의 표준화 점수(Z-score) 계산:
   - Z-score = (실제값 - 평균) / 표준편차
4. Z-score가 2 이상인 생산 건 찾기 (매우 높은 생산량)



In [ ]:
# 1. 설비별 평균 생산량을 계산하여 새 컬럼으로 추가 (transform)
production_df['설비별_생산량_평균'] = production_df.groupby('equipment_id')['actual_quantity'].transform('mean').round(2)
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,work_order_no,lot_no,operator_id,shift,created_at,updated_at,defect_rate,목표_달성률,year-month,설비별_생산량_평균
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,...,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,4.94,83.51,2024-01,107.49
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,...,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,7.69,93.98,2024-01,107.49
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,...,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.22,90.60,2024-01,120.83
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,...,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.17,92.00,2024-01,120.83
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,...,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,5.43,104.88,2024-01,120.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,...,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.36,96.00,2024-03,108.64
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,...,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,16.15,95.59,2024-03,108.64
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,...,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.50,95.24,2024-03,108.64
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,...,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,16.53,84.62,2024-03,96.09


In [140]:
# 2. 설비별 표준편차를 계산하여 새 컬럼으로 추가
production_df['설비별_생산량_표준편차'] = production_df.groupby('equipment_id')['actual_quantity'].transform('std').round(2)
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,lot_no,operator_id,shift,created_at,updated_at,defect_rate,목표_달성률,year-month,설비별_생산량_평균,설비별_생산량_표준편차
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,...,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,4.94,83.51,2024-01,107.49,20.02
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,...,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,7.69,93.98,2024-01,107.49,20.02
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,...,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.22,90.60,2024-01,120.83,22.11
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,...,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.17,92.00,2024-01,120.83,22.11
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,...,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,5.43,104.88,2024-01,120.83,22.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,...,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.36,96.00,2024-03,108.64,20.20
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,...,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,16.15,95.59,2024-03,108.64,20.20
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,...,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.50,95.24,2024-03,108.64,20.20
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,...,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,16.53,84.62,2024-03,96.09,16.58


In [144]:
# 3. 각 생산 건의 표준화 점수(Z-score) 계산:
#    - Z-score = (실제값 - 평균) / 표준편차
production_df['Z-score'] = ((production_df['actual_quantity'] - production_df['설비별_생산량_평균'])/production_df['설비별_생산량_표준편차']).round(4)
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,operator_id,shift,created_at,updated_at,defect_rate,목표_달성률,year-month,설비별_생산량_평균,설비별_생산량_표준편차,Z-score
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,...,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,4.94,83.51,2024-01,107.49,20.02,-1.3232
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,...,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,7.69,93.98,2024-01,107.49,20.02,-1.4730
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,...,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.22,90.60,2024-01,120.83,22.11,0.6409
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,...,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,2.17,92.00,2024-01,120.83,22.11,-1.3039
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,...,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,5.43,104.88,2024-01,120.83,22.11,0.3695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,...,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.36,96.00,2024-03,108.64,20.20,1.7505
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,...,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,16.15,95.59,2024-03,108.64,20.20,1.0574
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,...,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,17.50,95.24,2024-03,108.64,20.20,-1.4178
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,...,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,16.53,84.62,2024-03,96.09,16.58,1.5024


In [145]:
# 4. Z-score가 2 이상인 생산 건 찾기 (매우 높은 생산량)
production_df[production_df['Z-score'] >= 2]

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,operator_id,shift,created_at,updated_at,defect_rate,목표_달성률,year-month,설비별_생산량_평균,설비별_생산량_표준편차,Z-score


---
## 문제 10: 종합 설비 성능 대시보드

**시나리오**: 설비별 성능을 종합적으로 평가하는 대시보드 데이터를 만드세요.

**요구사항**:
1. 설비별로 다음 지표 모두 계산:
   - 생산 건수
   - 총 생산량
   - 평균 생산량
   - 생산량 표준편차
   - 평균 불량률
   - 평균 사이클 타임
   - 평균 목표 달성률
2. 성능 점수 계산 (사용자 정의):
   - 성능점수 = (평균생산량 / 평균사이클타임) * (100 - 평균불량률)
3. 성능 점수 기준 순위 매기기
4. 성능 점수 상위 5개 설비 출력



In [149]:
# 1. 설비별로 다음 지표 모두 계산:
#    - 생산 건수
#    - 총 생산량
#    - 평균 생산량
#    - 생산량 표준편차
#    - 평균 불량률
#    - 평균 사이클 타임
#    - 평균 목표 달성률
production_agg = production_df.groupby('equipment_id').agg(생산_건수=('production_id', 'count'),
                                                           총_생산량=('actual_quantity', 'sum'),
                                                           평균_생산량=('actual_quantity', 'mean'),
                                                           생산량_표준편차=('actual_quantity', 'std'),
                                                           평균_불량률=('defect_rate', 'mean'),
                                                           평균_사이클_타임=('cycle_time', 'mean'),
                                                           평균_목표_달성률=('목표_달성률', 'mean')).round(2)
production_agg

,생산_건수,총_생산량,평균_생산량,생산량_표준편차,평균_불량률,평균_사이클_타임,평균_목표_달성률
equipment_id,,,,,,,
ASM-001,234,22485,96.09,16.58,12.12,94.43,83.72
INJ-001,262,28163,107.49,20.02,10.76,71.71,93.58
INJ-002,430,51958,120.83,22.11,8.70,77.83,103.69
PRESS-001,468,52069,111.26,20.69,9.91,73.57,96.67
PRESS-002,478,51929,108.64,20.20,10.68,72.56,94.70


In [153]:
# 2. 성능 점수 계산 (사용자 정의):
#    - 성능점수 = (평균생산량 / 평균사이클타임) * (100 - 평균불량률)
production_agg['성능점수'] = ((production_agg['평균_생산량']/production_agg['평균_사이클_타임'])*(100 - production_agg['평균_불량률'])).round(2)
production_agg

,생산_건수,총_생산량,평균_생산량,생산량_표준편차,평균_불량률,평균_사이클_타임,평균_목표_달성률,성능점수
equipment_id,,,,,,,,
ASM-001,234,22485,96.09,16.58,12.12,94.43,83.72,89.42
INJ-001,262,28163,107.49,20.02,10.76,71.71,93.58,133.77
INJ-002,430,51958,120.83,22.11,8.70,77.83,103.69,141.74
PRESS-001,468,52069,111.26,20.69,9.91,73.57,96.67,136.24
PRESS-002,478,51929,108.64,20.20,10.68,72.56,94.70,133.73


In [154]:
# 3. 성능 점수 기준 순위 매기기
production_agg.sort_values('성능점수', ascending=False)

,생산_건수,총_생산량,평균_생산량,생산량_표준편차,평균_불량률,평균_사이클_타임,평균_목표_달성률,성능점수
equipment_id,,,,,,,,
INJ-002,430,51958,120.83,22.11,8.70,77.83,103.69,141.74
PRESS-001,468,52069,111.26,20.69,9.91,73.57,96.67,136.24
INJ-001,262,28163,107.49,20.02,10.76,71.71,93.58,133.77
PRESS-002,478,51929,108.64,20.20,10.68,72.56,94.70,133.73
ASM-001,234,22485,96.09,16.58,12.12,94.43,83.72,89.42


In [161]:
# 4. 성능 점수 상위 5개 설비 출력
production_agg.sort_values('성능점수', ascending=False).iloc[:5].index

Index(['INJ-002', 'PRESS-001', 'INJ-001', 'PRESS-002', 'ASM-001'], dtype='object', name='equipment_id')

---
## 수고하셨습니다!

### 학습 체크리스트
- [ ] groupby로 단일/다중 그룹화
- [ ] agg로 여러 집계 함수 동시 적용
- [ ] 명확한 컬럼명으로 집계 결과 생성
- [ ] pivot_table로 행/열 구조 변환
- [ ] crosstab으로 교차표 생성
- [ ] value_counts로 빈도 계산
- [ ] transform으로 그룹별 계산 결과 추가
- [ ] 복합 지표 계산 및 성능 평가

